In [1]:
import numpy as np 
import string 
from nltk.corpus import stopwords

In [2]:
def softmax(x): 
	e_x = np.exp(x - np.max(x)) 
	return e_x / e_x.sum() 

In [3]:
class word2vec(object): 
	def __init__(self): 
		self.N = 10
		self.X_train = [] 
		self.y_train = [] 
		self.window_size = 2
		self.alpha = 0.001
		self.words = [] 
		self.word_index = {} 

	def initialize(self,V,data): 
		self.V = V 
		self.W = np.random.uniform(-0.8, 0.8, (self.V, self.N)) 
		self.W1 = np.random.uniform(-0.8, 0.8, (self.N, self.V)) 
		
		self.words = data 
		for i in range(len(data)): 
			self.word_index[data[i]] = i 

	
	def feed_forward(self,X): 
		self.h = np.dot(self.W.T,X).reshape(self.N,1) 
		self.u = np.dot(self.W1.T,self.h) 
		#print(self.u) 
		self.y = softmax(self.u) 
		return self.y 
		
	def backpropagate(self,x,t): 
		e = self.y - np.asarray(t).reshape(self.V,1) 
		# e.shape is V x 1 
		dLdW1 = np.dot(self.h,e.T) 
		X = np.array(x).reshape(self.V,1) 
		dLdW = np.dot(X, np.dot(self.W1,e).T) 
		self.W1 -=  self.alpha*dLdW1 
		self.W -= self.alpha*dLdW 
		
	def train(self,epochs): 
		for x in range(1,epochs+1):		 
			self.loss = 0
			for j in range(len(self.X_train)): 
				self.feed_forward(self.X_train[j]) 
				self.backpropagate(self.X_train[j],self.y_train[j]) 
				C = 0
				for m in range(self.V): 
					if(self.y_train[j][m]): 
						self.loss += -1*self.u[m][0] 
						C += 1
				self.loss += C*np.log(np.sum(np.exp(self.u))) 
			print("epoch ",x, " loss = ",self.loss) 
			self.alpha *= 1/( (1+self.alpha*x) ) 
			
	def predict(self,word,number_of_predictions): 
		if word in self.words: 
			index = self.word_index[word] 
			X = [0 for i in range(self.V)] 
			X[index] = 1
			prediction = self.feed_forward(X) 
			output = {} 
			for i in range(self.V): 
				output[prediction[i][0]] = i 
			
			top_context_words = [] 
			for k in sorted(output,reverse=True): 
				top_context_words.append(self.words[output[k]]) 
				if(len(top_context_words)>=number_of_predictions): 
					break
	
			return top_context_words 
		else: 
			print("Word not found in dicitonary") 

In [4]:
def preprocessing(corpus): 
	stop_words = set(stopwords.words('english'))	 
	training_data = [] 
	sentences = corpus.split(".") 
	for i in range(len(sentences)): 
		sentences[i] = sentences[i].strip() 
		sentence = sentences[i].split() 
		x = [word.strip(string.punctuation) for word in sentence if word not in stop_words] 
		x = [word.lower() for word in x] 
		training_data.append(x) 
	return training_data 

In [5]:
def prepare_data_for_training(sentences,w2v): 
	data = {} 
	for sentence in sentences: 
		for word in sentence: 
			if word not in data: 
				data[word] = 1
			else: 
				data[word] += 1
	V = len(data) 
	data = sorted(list(data.keys())) 
	vocab = {} 
	for i in range(len(data)): 
		vocab[data[i]] = i 
	
	for sentence in sentences: 
		for i in range(len(sentence)): 
			center_word = [0 for x in range(V)] 
			center_word[vocab[sentence[i]]] = 1
			context = [0 for x in range(V)] 
			
			for j in range(i-w2v.window_size,i+w2v.window_size): 
				if i!=j and j>=0 and j<len(sentence): 
					context[vocab[sentence[j]]] += 1
			w2v.X_train.append(center_word) 
			w2v.y_train.append(context) 
	w2v.initialize(V,data) 

	return w2v.X_train,w2v.y_train 

In [6]:
corpus = "The earth revolves around the sun. The moon revolves around the earth"
epochs = 1000

In [7]:
training_data = preprocessing(corpus) 

In [8]:
training_data

[['the', 'earth', 'revolves', 'around', 'sun'],
 ['the', 'moon', 'revolves', 'around', 'earth']]

In [9]:
w2v = word2vec() 

In [10]:
prepare_data_for_training(training_data,w2v) 

([[0, 0, 0, 0, 0, 1],
  [0, 1, 0, 0, 0, 0],
  [0, 0, 0, 1, 0, 0],
  [1, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 1, 0],
  [0, 0, 0, 0, 0, 1],
  [0, 0, 1, 0, 0, 0],
  [0, 0, 0, 1, 0, 0],
  [1, 0, 0, 0, 0, 0],
  [0, 1, 0, 0, 0, 0]],
 [[0, 1, 0, 0, 0, 0],
  [0, 0, 0, 1, 0, 1],
  [1, 1, 0, 0, 0, 1],
  [0, 1, 0, 1, 1, 0],
  [1, 0, 0, 1, 0, 0],
  [0, 0, 1, 0, 0, 0],
  [0, 0, 0, 1, 0, 1],
  [1, 0, 1, 0, 0, 1],
  [0, 1, 1, 1, 0, 0],
  [1, 0, 0, 1, 0, 0]])

In [11]:
w2v.train(epochs) 

epoch  1  loss =  49.86954914203421
epoch  2  loss =  49.735911699717754
epoch  3  loss =  49.60375846938555
epoch  4  loss =  49.473197244857445
epoch  5  loss =  49.34432936673736
epoch  6  loss =  49.21724927240098
epoch  7  loss =  49.09204412424414
epoch  8  loss =  48.96879351899138
epoch  9  loss =  48.84756927913462
epoch  10  loss =  48.72843532589865
epoch  11  loss =  48.611447631560885
epoch  12  loss =  48.496654247535105
epoch  13  loss =  48.38409540340127
epoch  14  loss =  48.27380367104611
epoch  15  loss =  48.16580418729182
epoch  16  loss =  48.06011492783421
epoch  17  loss =  47.956747024981354
epoch  18  loss =  47.85570512156767
epoch  19  loss =  47.75698775349437
epoch  20  loss =  47.660587753591116
epoch  21  loss =  47.5664926698782
epoch  22  loss =  47.47468519180412
epoch  23  loss =  47.38514357861081
epoch  24  loss =  47.29784208461159
epoch  25  loss =  47.21275137682831
epoch  26  loss =  47.12983894110381
epoch  27  loss =  47.04906947346334
epoch

In [12]:
w2v.predict("around",3)

['revolves', 'around', 'the']

In [13]:
w2v.predict("around",5)

['revolves', 'around', 'the', 'sun', 'moon']

In [14]:
w2v.predict("sun",3)

['around', 'sun', 'earth']

In [15]:
w2v.predict("earth",3)

['the', 'moon', 'around']

In [16]:
w2v.predict("jupiter",3)

Word not found in dicitonary
